In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from itertools import chain

from TPR_utils import TPR, build_D, DecodedTPR2Tree
from models import *
from data import BinaryT2TDataset

In [9]:
batch_size = 4
max_depth = 6
d_filler = 32
d_role = 64


In [26]:
train_data = BinaryT2TDataset('./data_files/nc_pat/car_cdr_rcons/train.xy', max_depth=max_depth, device='cpu') #, filter=lambda x:x=='rcons')
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

valid_data = BinaryT2TDataset('./data_files/nc_pat/car_cdr_rcons/dev.xy', max_depth=max_depth, ind2vocab=train_data.ind2vocab, device='cpu') #, filter=lambda x:x=='rcons')
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)

data rows loaded: 324
data rows loaded: 40


In [14]:
tpr = TPR(num_fillers=len(train_data.ind2vocab), num_roles=2**max_depth-1, d_filler=d_filler, d_role=d_role)
D_l, D_r = build_D(tpr.role_emb)
E_l, E_r = build_E(tpr.role_emb)

lambda car: 

In [45]:
len(train_data.ind2vocab)

31

In [17]:
D_l.shape

torch.Size([64, 64])

In [76]:
root_role = tpr.role_emb.weight[0]

In [78]:
root_role

tensor([ 0.0930,  0.0784, -0.1213, -0.1451,  0.1395,  0.0109, -0.2583, -0.0918,
        -0.0776, -0.2019,  0.1036, -0.2283,  0.1306,  0.0228, -0.0522, -0.0193,
         0.1485,  0.2698, -0.0237,  0.2197,  0.0413, -0.1721, -0.0050, -0.0712,
        -0.2527,  0.2112, -0.0607,  0.1920, -0.1990,  0.1063, -0.0322,  0.1279,
         0.1621, -0.1533, -0.1395,  0.0698, -0.0735,  0.1017,  0.1976,  0.0513,
        -0.0532, -0.1838,  0.1341, -0.1785, -0.0046, -0.1436, -0.0470,  0.0471,
        -0.0298, -0.0126, -0.0025,  0.0256,  0.0534,  0.0097,  0.0339, -0.0665,
         0.0781,  0.0103,  0.0823, -0.0029,  0.1055, -0.1691, -0.0328, -0.1412],
       grad_fn=<SelectBackward0>)

In [83]:
print(D_r @ E_r @ root_role)
print(D_l @ E_l @ root_role)

tensor([ 0.0930,  0.0784, -0.1213, -0.1451,  0.1395,  0.0109, -0.2583, -0.0918,
        -0.0776, -0.2019,  0.1036, -0.2283,  0.1306,  0.0228, -0.0522, -0.0193,
         0.1485,  0.2698, -0.0237,  0.2197,  0.0413, -0.1721, -0.0050, -0.0712,
        -0.2527,  0.2111, -0.0607,  0.1920, -0.1990,  0.1063, -0.0322,  0.1279,
         0.1621, -0.1533, -0.1395,  0.0698, -0.0735,  0.1017,  0.1976,  0.0513,
        -0.0532, -0.1838,  0.1341, -0.1785, -0.0046, -0.1436, -0.0470,  0.0471,
        -0.0298, -0.0126, -0.0025,  0.0256,  0.0534,  0.0097,  0.0339, -0.0665,
         0.0781,  0.0103,  0.0823, -0.0029,  0.1055, -0.1691, -0.0328, -0.1412],
       grad_fn=<MvBackward0>)
tensor([ 0.0930,  0.0784, -0.1213, -0.1451,  0.1395,  0.0109, -0.2583, -0.0918,
        -0.0776, -0.2019,  0.1036, -0.2283,  0.1306,  0.0228, -0.0522, -0.0193,
         0.1485,  0.2698, -0.0237,  0.2197,  0.0413, -0.1721, -0.0050, -0.0712,
        -0.2527,  0.2112, -0.0607,  0.1920, -0.1990,  0.1063, -0.0322,  0.1279,
         

In [84]:
print(D_l @ E_r @ root_role)
print(D_r @ E_l @ root_role)

tensor([ 3.0413e-08, -1.4204e-08, -1.6779e-08, -2.2919e-08,  2.7214e-08,
        -3.0917e-08,  2.1539e-08,  4.1112e-08, -6.2098e-09,  4.2360e-08,
         9.5209e-09,  4.4208e-08, -3.5445e-08, -5.0748e-09, -2.3751e-08,
        -8.8919e-09,  7.6503e-09, -2.6460e-08,  3.8534e-08, -3.3531e-08,
        -3.8955e-08, -1.7891e-08,  1.5992e-09, -3.0888e-08, -2.0969e-08,
        -8.5689e-10, -1.8192e-09,  3.6812e-08,  3.0505e-08,  2.0606e-08,
        -7.7411e-09, -2.0174e-08, -2.2087e-08,  1.1386e-08, -1.1883e-08,
         9.1152e-08, -2.3103e-08,  1.0035e-08,  6.9190e-09, -2.4429e-08,
        -3.4202e-08, -4.2551e-08,  1.6088e-08,  3.9973e-08,  4.1187e-08,
        -4.2665e-08,  2.9056e-08, -8.9846e-09, -9.3284e-09,  9.3652e-09,
        -1.3388e-08, -1.7880e-10, -6.6199e-09, -3.3671e-08,  3.5411e-09,
        -1.7860e-08,  4.6978e-08,  6.3333e-09,  2.8112e-09,  5.0766e-09,
        -5.2740e-08, -3.4955e-08, -6.5707e-10,  4.3593e-09],
       grad_fn=<MvBackward0>)
tensor([-2.1012e-08,  4.0701e-09,

In [88]:
print(E_l @ root_role)
print(tpr.role_emb.weight[1])

tensor([ 0.2382,  0.2243,  0.1132,  0.0084, -0.1034,  0.3020,  0.1480, -0.0428,
         0.3078,  0.0548,  0.1761, -0.0059,  0.0699, -0.0221, -0.0671, -0.2050,
         0.0545,  0.0152, -0.0622,  0.0017, -0.0545, -0.0446, -0.0242,  0.1690,
         0.1221, -0.0248,  0.0373, -0.0236,  0.0346,  0.0837, -0.1438,  0.1063,
         0.0086,  0.0257, -0.1918, -0.0548,  0.0895,  0.0393,  0.0715, -0.2458,
        -0.0075, -0.2638, -0.1241, -0.1486, -0.0818,  0.1526,  0.0292, -0.1327,
         0.0822, -0.0341, -0.0178, -0.1549, -0.0319,  0.2171,  0.1595,  0.0359,
        -0.0533, -0.0860,  0.2085,  0.0041,  0.0996,  0.1800,  0.0123,  0.0222],
       grad_fn=<MvBackward0>)
tensor([ 0.2382,  0.2243,  0.1132,  0.0084, -0.1034,  0.3020,  0.1480, -0.0428,
         0.3078,  0.0548,  0.1761, -0.0059,  0.0699, -0.0221, -0.0671, -0.2050,
         0.0545,  0.0152, -0.0622,  0.0017, -0.0545, -0.0446, -0.0242,  0.1690,
         0.1221, -0.0248,  0.0373, -0.0236,  0.0346,  0.0837, -0.1438,  0.1063,
         

In [91]:
D_l @ D_r @ E_r @ E_l @ root_role

tensor([ 0.0930,  0.0784, -0.1213, -0.1451,  0.1395,  0.0109, -0.2583, -0.0918,
        -0.0776, -0.2019,  0.1036, -0.2283,  0.1306,  0.0228, -0.0522, -0.0193,
         0.1485,  0.2698, -0.0237,  0.2197,  0.0413, -0.1721, -0.0050, -0.0712,
        -0.2527,  0.2112, -0.0607,  0.1920, -0.1990,  0.1063, -0.0322,  0.1279,
         0.1621, -0.1533, -0.1395,  0.0698, -0.0735,  0.1017,  0.1976,  0.0513,
        -0.0532, -0.1838,  0.1341, -0.1785, -0.0046, -0.1436, -0.0470,  0.0471,
        -0.0298, -0.0126, -0.0025,  0.0256,  0.0534,  0.0097,  0.0339, -0.0665,
         0.0781,  0.0103,  0.0823, -0.0029,  0.1055, -0.1691, -0.0328, -0.1412],
       grad_fn=<MvBackward0>)

In [92]:
print(D_r @ E_r)
print((D_r @ E_r) @ (D_r @ E_r))

tensor([[ 0.5839, -0.0399, -0.0623,  ...,  0.0228,  0.0304, -0.0501],
        [-0.0399,  0.5193, -0.0456,  ...,  0.0253, -0.0623,  0.1048],
        [-0.0623, -0.0456,  0.2755,  ...,  0.1049,  0.0470, -0.0446],
        ...,
        [ 0.0228,  0.0253,  0.1049,  ...,  0.3334, -0.0042,  0.0249],
        [ 0.0304, -0.0623,  0.0470,  ..., -0.0042,  0.3743, -0.0789],
        [-0.0501,  0.1048, -0.0446,  ...,  0.0249, -0.0789,  0.5666]])
tensor([[ 0.5839, -0.0399, -0.0623,  ...,  0.0228,  0.0304, -0.0501],
        [-0.0399,  0.5193, -0.0456,  ...,  0.0253, -0.0623,  0.1048],
        [-0.0623, -0.0456,  0.2755,  ...,  0.1049,  0.0470, -0.0446],
        ...,
        [ 0.0228,  0.0253,  0.1049,  ...,  0.3334, -0.0042,  0.0249],
        [ 0.0304, -0.0623,  0.0470,  ..., -0.0042,  0.3743, -0.0789],
        [-0.0501,  0.1048, -0.0446,  ...,  0.0249, -0.0789,  0.5666]])


In [99]:
torch.matrix_rank(D_r @ E_r)

tensor(31)

In [95]:
D_r.shape

torch.Size([64, 64])

In [101]:
torch.matrix_rank(tpr.role_emb.weight)

tensor(63)

In [102]:
tpr.role_emb.weight.shape

torch.Size([63, 64])

In [167]:
print('E_r.shape {}'.format(E_r.shape))

E_r.shape torch.Size([64, 64])


In [168]:
print(root_role)

tensor([ 0.0930,  0.0784, -0.1213, -0.1451,  0.1395,  0.0109, -0.2583, -0.0918,
        -0.0776, -0.2019,  0.1036, -0.2283,  0.1306,  0.0228, -0.0522, -0.0193,
         0.1485,  0.2698, -0.0237,  0.2197,  0.0413, -0.1721, -0.0050, -0.0712,
        -0.2527,  0.2112, -0.0607,  0.1920, -0.1990,  0.1063, -0.0322,  0.1279,
         0.1621, -0.1533, -0.1395,  0.0698, -0.0735,  0.1017,  0.1976,  0.0513,
        -0.0532, -0.1838,  0.1341, -0.1785, -0.0046, -0.1436, -0.0470,  0.0471,
        -0.0298, -0.0126, -0.0025,  0.0256,  0.0534,  0.0097,  0.0339, -0.0665,
         0.0781,  0.0103,  0.0823, -0.0029,  0.1055, -0.1691, -0.0328, -0.1412],
       grad_fn=<SelectBackward0>)


In [169]:
print('D_r @ E_r: \n{}'.format(D_r @ E_r))

D_r @ E_r: 
tensor([[ 0.5839, -0.0399, -0.0623,  ...,  0.0228,  0.0304, -0.0501],
        [-0.0399,  0.5193, -0.0456,  ...,  0.0253, -0.0623,  0.1048],
        [-0.0623, -0.0456,  0.2755,  ...,  0.1049,  0.0470, -0.0446],
        ...,
        [ 0.0228,  0.0253,  0.1049,  ...,  0.3334, -0.0042,  0.0249],
        [ 0.0304, -0.0623,  0.0470,  ..., -0.0042,  0.3743, -0.0789],
        [-0.0501,  0.1048, -0.0446,  ...,  0.0249, -0.0789,  0.5666]])


In [170]:
print('D_r @ E_r @ root_role == root_role ? {}'.format(torch.isclose(D_r @ E_r @ root_role, root_role).all()))

D_r @ E_r @ root_role == root_role ? True


In [171]:
print('D_l @ E_r @ root_role == 0 ? {}'.format(torch.isclose(D_l @ E_r @ root_role, torch.zeros_like(root_role), rtol=1).all()))

D_l @ E_r @ root_role == 0 ? False


In [172]:
print('D_l @ E_r: \n{}'.format(D_l @ E_r))

D_l @ E_r: 
tensor([[-2.6409e-08, -1.1797e-08, -2.3241e-09,  ..., -2.8238e-08,
         -3.2162e-08,  2.0257e-08],
        [ 3.5664e-10, -2.1051e-08,  2.2797e-09,  ..., -3.9010e-09,
          8.8812e-10, -3.5749e-08],
        [ 1.5282e-08, -7.7841e-09,  1.8504e-08,  ...,  1.9343e-08,
          2.7531e-08, -2.5500e-10],
        ...,
        [-5.4678e-09, -4.5274e-08,  2.2085e-08,  ...,  5.1012e-09,
          4.4758e-09, -7.0986e-09],
        [ 1.3274e-08,  4.7594e-09, -1.2373e-08,  ..., -1.0084e-08,
          1.9362e-08,  5.8964e-09],
        [ 8.4993e-09, -2.3226e-08,  5.1702e-09,  ...,  2.1577e-08,
          1.5970e-08, -1.9975e-08]])


In [174]:
print('E_r rank: {}'.format(torch.matrix_rank(E_r)))
print('D_r rank: {}'.format(torch.matrix_rank(D_r)))
print('D_r @ E_r rank: {}'.format(torch.matrix_rank(D_r @ E_r)))


E_r rank: 31
D_r rank: 31
D_r @ E_r rank: 31


In [194]:
D_l @ E_l @ E_l @ E_l @ E_l @ E_l @ E_l @ root_role

tensor([ 1.9212e-08,  3.5433e-08, -5.3675e-08, -4.2428e-08,  2.5903e-08,
         4.0175e-08,  2.4228e-08, -4.5179e-09,  6.8896e-09,  2.3810e-08,
         6.6187e-08, -1.7044e-08,  3.5585e-08, -9.1178e-08, -3.0079e-08,
         6.0685e-09, -2.5106e-08, -4.0212e-08, -3.1141e-08,  1.1014e-08,
         3.1421e-08,  2.1261e-08,  8.1320e-09,  1.9499e-09,  9.9952e-10,
        -2.2876e-08, -5.3023e-08, -5.4134e-08, -2.0643e-08, -2.1224e-08,
         2.7631e-08,  4.6476e-08,  1.1268e-07, -4.8461e-09,  8.5104e-08,
         1.2843e-08, -1.0388e-08,  1.4189e-08,  5.1149e-08,  1.4210e-08,
         3.2144e-08, -5.6159e-08,  2.8521e-08,  1.1968e-08,  2.3145e-08,
        -1.7128e-08, -2.5224e-08,  5.3672e-08,  4.1943e-08,  3.5132e-08,
        -6.8100e-09, -1.6013e-08,  7.0402e-08, -8.5101e-08, -1.8713e-09,
        -8.2456e-08,  6.4158e-08,  2.8502e-08, -1.8875e-08,  5.7050e-08,
         2.2675e-08,  3.4870e-08, -1.2620e-08, -4.3095e-08],
       grad_fn=<MvBackward0>)